In [23]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
tf.__version__, keras.__version__

('2.0.0', '2.2.4-tf')

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [4]:
X_train_full.shape, X_test.shape, X_train_full.dtype

((60000, 28, 28), (10000, 28, 28), dtype('uint8'))

In [5]:
# Split into training and validation
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [6]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
"Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [7]:
class_names[y_train[0]]

'Coat'

In [12]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_4 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.layers[1].name

'dense_3'

In [15]:
print("weights for layer 1:", model.get_weights()[0].shape)
print("bias for layer 1:", model.get_weights()[1].shape)

weights for layer 1: (784, 300)
bias for layer 1: (300,)


In [16]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [17]:
history = model.fit(X_train, y_train, epochs=30, 
                    validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/30
55000/55000 [==============================] - 3s 59us/sample - loss: 0.7047 - accuracy: 0.7717 - val_loss: 0.5027 - val_accuracy: 0.8240
Epoch 2/30
55000/55000 [==============================] - 3s 51us/sample - loss: 0.4823 - accuracy: 0.8325 - val_loss: 0.4787 - val_accuracy: 0.8282
Epoch 3/30
55000/55000 [==============================] - 3s 52us/sample - loss: 0.4396 - accuracy: 0.8465 - val_loss: 0.4147 - val_accuracy: 0.8594
Epoch 4/30
55000/55000 [==============================] - 3s 54us/sample - loss: 0.4121 - accuracy: 0.8554 - val_loss: 0.3877 - val_accuracy: 0.8672
Epoch 5/30
55000/55000 [==============================] - 3s 52us/sample - loss: 0.3927 - accuracy: 0.8635 - val_loss: 0.3773 - val_accuracy: 0.8702
Epoch 6/30
55000/55000 [==============================] - 3s 54us/sample - loss: 0.3761 - accuracy: 0.8686 - val_loss: 0.3706 - val_accuracy: 0.8688
Epoch 7/30
55000/55000 [==============================] -

In [19]:
#Save .h5 model and parameters
model.save("model0.h5")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
pd.DataFrame(history.history)

In [ ]:
X_new = X_test[:3]
y_proba = model.predict(X_new)

In [ ]:
#Probability of each class
y_proba.round(2)

In [ ]:
y_pred = model.predict_classes(X_new)
y_pred

In [ ]:
np.array(class_names)[y_pred]

## Regression

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
X_train_full, y_train_full)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled.shape

In [ ]:
model1 = keras.models.Sequential([
keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
keras.layers.Dense(1)])
model1.compile(loss="mean_squared_error", optimizer="sgd")

In [ ]:
history1 = model1.fit(X_train_scaled, y_train, epochs=20, 
                    validation_data=(X_valid_scaled, y_valid))

In [ ]:
mse_test = model1.evaluate(X_test_scaled, y_test, verbose=0)
mse_test

In [ ]:
X_new = X_test[:3] # pretend these are new instances
y_pred = model1.predict(X_new)

In [ ]:
y_pred

### Wide and Deep NN

In [ ]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model2 = keras.models.Model(inputs=[input_], outputs=[output])

In [ ]:
input_A = keras.layers.Input(shape=[5])
input_B = keras.layers.Input(shape=[6])
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1)(concat)
model3 = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

In [ ]:
model3.compile(loss="mse", optimizer="sgd")

In [ ]:
X_train_A, X_train_B = X_train_scaled[:, :5], X_train_scaled[:, 2:]
X_valid_A, X_valid_B = X_valid_scaled[:, :5], X_valid_scaled[:, 2:]
X_test_A, X_test_B = X_test_scaled[:, :5], X_test_scaled[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [ ]:
history3 = model3.fit((X_train_A, X_train_B), y_train, epochs=20, 
                    validation_data=((X_valid_A, X_valid_B), y_valid))

In [ ]:
mse_test = model3.evaluate((X_test_A, X_test_B), y_test, verbose=0)
y_pred = model3.predict((X_new_A, X_new_B))
mse_test

### Subclassing API

In [ ]:
class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # handles standard args (e.g., name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
model4 = WideAndDeepModel()

In [ ]:
model4.compile(loss="mse", optimizer="adam")

In [ ]:
history4 = model4.fit((X_train_A, X_train_B), y_train, epochs=20, 
                    validation_data=((X_valid_A, X_valid_B), y_valid))

In [ ]:
model4.summary()

In [ ]:
class MyModel(keras.Model):

    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # Define your layers here.
        self.dense_1 = keras.layers.Dense(32, activation='relu')
        self.dense_2 = keras.layers.Dense(num_classes, activation='relu')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.dense_1(inputs)
        return self.dense_2(x)

    def compute_output_shape(self, input_shape):
        # You need to override this function if you want to use the subclassed model
        # as part of a functional-style model.
        # Otherwise, this method is optional.
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)


# Instantiates the subclassed model.
model5 = MyModel(num_classes=10)

# The compile step specifies the training configuration.
model5.compile(keras.optimizers.RMSprop(0.001),
              loss='mse',
              metrics=['mse'])

data = X_train_scaled; labels=y_train
# Trains for 5 epochs.
model5.fit(data, labels, batch_size=32, epochs=5)

### Cross Validation

In [ ]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    options = {"input_shape": input_shape}
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu", **options))
        options = {}
    model.add(keras.layers.Dense(1, **options))
    optimizer = keras.optimizers.SGD(learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [ ]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [ ]:
y_pred

### Randomized CV

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
                    "n_hidden": [0, 1, 2, 3],
                    "n_neurons": np.arange(1, 100),
                    "learning_rate": reciprocal(3e-4, 3e-2),
                    }
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=3, cv=3)
rnd_search_cv.fit(X_train_scaled, y_train, epochs=10, 
                  validation_data=(X_valid_scaled, y_valid), 
                  callbacks=[keras.callbacks.EarlyStopping(patience=3)],
                    verbose=0)

In [ ]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)
model = rnd_search_cv.best_estimator_.model

### Tensors and Operations

In [20]:
tf.constant([[1., 2., 3.], [4., 5., 6.]]) # matrix

<tf.Tensor: id=169903, shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [21]:
tf.constant(42)

<tf.Tensor: id=169904, shape=(), dtype=int32, numpy=42>

In [22]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print(t.shape, t.dtype)

(2, 3) <dtype: 'float32'>


## Convolutional NN

In [24]:
from sklearn.datasets import load_sample_image

# Load sample images
china = load_sample_image("china.jpg") / 255
flower = load_sample_image("flower.jpg") / 255
images = np.array([china, flower])
batch_size, height, width, channels = images.shape

In [ ]:
from functools import partial
DefaultConv2D = partial(keras.layers.Conv2D,
    kernel_size=3, activation='relu', padding="SAME")
    model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),
])

### Using Pretrained Models

In [27]:
model = keras.applications.resnet50.ResNet50(weights="imagenet")

102973440/102967424 [==============================] - 66s 1us/step


In [28]:
# ResNet-50 requires input of size 224 X 224
images_resized = tf.image.resize(images, [224, 224])

In [29]:
# PReprocessing for ResNet
inputs = keras.applications.resnet50.preprocess_input(images_resized * 255)

In [30]:
Y_proba = model.predict(inputs)

In [32]:
top_K = keras.applications.resnet50.decode_predictions(Y_proba, top=5)
for image_index in range(len(images)):
    print("Image #{}".format(image_index))
    for class_id, name, y_proba in top_K[image_index]:
        print(" {} - {:12s} {:.2f}%".format(class_id, name, y_proba * 100))
    print()

Image #0
 n02825657 - bell_cote    82.35%
 n03877845 - palace       6.75%
 n03781244 - monastery    4.19%
 n02980441 - castle       3.16%
 n03028079 - church       1.00%

Image #1
 n03530642 - honeycomb    49.81%
 n13040303 - stinkhorn    33.96%
 n02206856 - bee          4.35%
 n11939491 - daisy        3.23%
 n12985857 - coral_fungus 2.66%



In [84]:
def BinarySearch(arr, val):
    first = 0
    last = len(arr) - 1
    mid = (first + last)//2
    
    if last < first: return -1
    elif arr[mid] < val:
        #first = mid+1
        return BinarySearch(arr[mid+1:last+1], val)
    elif arr[mid] > val: 
        #last = mid - 1
        return BinarySearch(arr[first:mid], val)
    else:
        if arr[mid] == val: return 1
        
result = BinarySearch(arr[first:mid], val)
        

In [95]:
print(BinarySearch([1,2,3,4,5], 2))

1


In [93]:
# Python Program for recursive binary search. 

# Returns index of x in arr if present, else -1 
def binarySearch (arr, l, r, x): 

	# Check base case 
	if r >= l: 

		mid = l + (r - l)//2

		# If element is present at the middle itself 
		if arr[mid] == x: 
			return mid 
		
		# If element is smaller than mid, then it 
		# can only be present in left subarray 
		elif arr[mid] > x: 
			return binarySearch(arr, l, mid-1, x) 

		# Else the element can only be present 
		# in right subarray 
		else: 
			return binarySearch(arr, mid + 1, r, x) 

	else: 
		# Element is not present in the array 
		return -1

In [102]:
a = np.arange(10).reshape(5,1,2)

In [103]:
a.shape

(5, 1, 2)

In [105]:
np.squeeze(a).shape

(5, 2)